# Understanding the Paillier Cryptosystem

2023, Thomas Liebig
[Compare my previous notes (German)](https://www-ai.cs.tu-dortmund.de/LEHRE/FACHPROJEKT/WS1516/download/paillier.pdf)

## Key Generation

In [1]:
p=5
q=3

In [2]:
import math

n=p*q
phi=(p-1)*(q-1) # Eulers Phi
g=n+1           # Common choice of g

lambda1=math.lcm(p-1,q-1) # Carmichael number

## Encryption
$$
\begin{aligned}
\mathcal{E}_g: \mathbb{Z}_n \times \mathbb{Z}_n^* & \rightarrow \mathbb{Z}_{n^2}^* \\
\mathcal{E}_g(x, y) & \rightarrow g^x \cdot y^n \bmod n^2 \\
\mathcal{E}_g(m, r) & \rightarrow g^m \cdot r^n \bmod n^2
\end{aligned}
$$

## Decryption
$$m=L(c^\lambda mod n^2)* L^{-1}(g^\lambda \text{mod} n^2)$$
mit $L(x)=(x-1)/n$

In [3]:
import numpy as np
from math import gcd

A = np.zeros((phi, n))

j = 0

names=[]

for i in range(phi):
    j += 1
    while gcd(j, n) != 1:
        j += 1
    names.append(j)
    for m in range(n):
        A[i, m] = (pow(g, m, pow(n, 2)) * pow(j, n, pow(n, 2))) % pow(n, 2)


In [4]:
def printMatrix(s,names):

    # Do heading
    print("     ", end="")
    for j in range(len(s[0])):
        print("%5d " % j, end="")
    print()
    print("     ", end="")
    for j in range(len(s[0])):
        print("------", end="")
    print()
    # Matrix contents
    for i in range(len(s)):
        print("%3d |" % (names[i]), end="") # Row nums
        for j in range(len(s[0])):
            print("%5d " % (s[i][j]), end="")
        print()  


### Inspect matrix of all ciphertexts A

In [5]:
printMatrix(A,names)

         0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
     ------------------------------------------------------------------------------------------
  1 |    1    16    31    46    61    76    91   106   121   136   151   166   181   196   211 
  2 |  143    38   158    53   173    68   188    83   203    98   218   113     8   128    23 
  4 |  199    34    94   154   214    49   109   169     4    64   124   184    19    79   139 
  7 |  118    88    58    28   223   193   163   133   103    73    43    13   208   178   148 
  8 |  107   137   167   197     2    32    62    92   122   152   182   212    17    47    77 
 11 |   26   191   131    71    11   176   116    56   221   161   101    41   206   146    86 
 13 |   82   187    67   172    52   157    37   142    22   127     7   112   217    97   202 
 14 |  224   209   194   179   164   149   134   119   104    89    74    59    44    29    14 


Observe the homomorphic property: Take two numbers from two columns $a$ and $b$, multiply these number and observe that their product is in the column $a+b$.

### Inspect Decryption Steps I: Inspect matrix $A^\lambda$

In [6]:
A_l=np.empty(A.shape)
for i in range(0,A.shape[0]):
        for j in range(0,A.shape[1]):
            A_l[i][j]=pow(int(A[i,j]),lambda1,pow(n, 2))
printMatrix(A_l,names)

         0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
     ------------------------------------------------------------------------------------------
  1 |    1    61   121   181    16    76   136   196    31    91   151   211    46   106   166 
  2 |    1    61   121   181    16    76   136   196    31    91   151   211    46   106   166 
  4 |    1    61   121   181    16    76   136   196    31    91   151   211    46   106   166 
  7 |    1    61   121   181    16    76   136   196    31    91   151   211    46   106   166 
  8 |    1    61   121   181    16    76   136   196    31    91   151   211    46   106   166 
 11 |    1    61   121   181    16    76   136   196    31    91   151   211    46   106   166 
 13 |    1    61   121   181    16    76   136   196    31    91   151   211    46   106   166 
 14 |    1    61   121   181    16    76   136   196    31    91   151   211    46   106   166 


### Inspect Decryption Steps II: Inspect matrix $L(A^\lambda)\text{ mod }n^2$

In [7]:
A_l=np.empty(A.shape)
for i in range(0,A.shape[0]):
        for j in range(0,A.shape[1]):
            A_l[i][j]=(pow(int(A[i,j]),lambda1,pow(n, 2))-1)/n
printMatrix(A_l,names)

         0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
     ------------------------------------------------------------------------------------------
  1 |    0     4     8    12     1     5     9    13     2     6    10    14     3     7    11 
  2 |    0     4     8    12     1     5     9    13     2     6    10    14     3     7    11 
  4 |    0     4     8    12     1     5     9    13     2     6    10    14     3     7    11 
  7 |    0     4     8    12     1     5     9    13     2     6    10    14     3     7    11 
  8 |    0     4     8    12     1     5     9    13     2     6    10    14     3     7    11 
 11 |    0     4     8    12     1     5     9    13     2     6    10    14     3     7    11 
 13 |    0     4     8    12     1     5     9    13     2     6    10    14     3     7    11 
 14 |    0     4     8    12     1     5     9    13     2     6    10    14     3     7    11 


In [8]:
def gcdExtended(a, b): 
    if a == 0 :  
       return b,0,1
             
    gcd,x1,y1 = gcdExtended(b % a, a) 
     
    x = y1 - (b//a) * x1 
    y = x1 
     
    return gcd,x,y


### Inspect Decryption Steps III: Inspect matrix $L(A^\lambda)/L(g^\lambda)\text{ mod }n$

In [9]:
Lg = (pow(g,lambda1,pow(n,2))-1)/n
Lg_inv = (gcdExtended(Lg,n)[1]+n) % n

#((modpower(A,lambda,n^2)-1)/n) * Lg_inv) %% n

A_i=np.empty(A.shape)
for i in range(0,A.shape[0]):
        for j in range(0,A.shape[1]):
            A_i[i][j]=(((pow(int(A[i,j]),lambda1,pow(n, 2))-1)/n) * Lg_inv ) % n
printMatrix(A_i,names)

         0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
     ------------------------------------------------------------------------------------------
  1 |    0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
  2 |    0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
  4 |    0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
  7 |    0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
  8 |    0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
 11 |    0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
 13 |    0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
 14 |    0     1     2     3     4     5     6     7     8     9    10    11    12    13    14 
